<a href="https://colab.research.google.com/github/Pravin98651/FFCS-Helper-AI-Powered-Timetable-Faculty-Selection-for-FFCS/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files, drive
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor


drive.mount('/content/drive')


model_path = '/content/drive/My Drive/FFCS_Helper_Models/'
os.makedirs(model_path, exist_ok=True)


uploaded = files.upload()
df = pd.read_csv("1faculty_data.csv")
print("✅ Data Loaded Successfully")
df.head()

encoder = OneHotEncoder(sparse_output=False)
dept_encoded = encoder.fit_transform(df[['Department']])
dept_columns = encoder.get_feature_names_out(['Department'])
df_dept = pd.DataFrame(dept_encoded, columns=dept_columns)
df = pd.concat([df.drop('Department', axis=1), df_dept], axis=1)

X = df.drop(columns=['EmpId'])
y = df[['Teaching', 'Lenient', 'DA and Quiz', 'Marking']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print("✅ Data Preprocessing Complete")


with open(os.path.join(model_path, 'scaler.pkl'), 'wb') as file:
    pickle.dump(scaler, file)
print("✅ Scaler Saved to Google Drive")


Mounted at /content/drive


Saving 1faculty_data.csv to 1faculty_data.csv
✅ Data Loaded Successfully
✅ Data Preprocessing Complete
✅ Scaler Saved to Google Drive


In [2]:

models = {}
categories = ['Teaching', 'Lenient', 'DA and Quiz', 'Marking']
for category in categories:
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train[category])
    models[category] = model


for category, model in models.items():
    with open(os.path.join(model_path, f"{category}_model.pkl"), 'wb') as file:
        pickle.dump(model, file)
print(f"✅ Models Saved Successfully in Google Drive: {model_path}")


✅ Models Saved Successfully in Google Drive: /content/drive/My Drive/FFCS_Helper_Models/


In [3]:

models = {}
for category in categories:
    with open(os.path.join(model_path, f"{category}_model.pkl"), 'rb') as file:
        models[category] = pickle.load(file)

with open(os.path.join(model_path, 'scaler.pkl'), 'rb') as file:
    scaler = pickle.load(file)

print("✅ Models & Scaler Loaded Successfully")


✅ Models & Scaler Loaded Successfully


In [4]:
def predict_best_faculty(faculty_list, student_prefs):
    """
    Predicts and ranks faculties based on rating categories and student preferences.
    Returns:
    - Best faculty for each category (Teaching, Lenient, DA and Quiz, Marking).
    - Overall best faculty based on student preferences.
    - Full ranked list of faculties.
    """
    filtered_faculty = df[df['EmpId'].isin(faculty_list)].copy()

    if filtered_faculty.empty:
        return {"message": "Prediction not available. No faculty found in the given list."}

    faculty_features = filtered_faculty.drop(columns=['EmpId'])
    faculty_features_scaled = scaler.transform(faculty_features)

    predicted_scores = {cat: models[cat].predict(faculty_features_scaled) for cat in categories}

    for cat in categories:
        filtered_faculty[f"Predicted_{cat}"] = predicted_scores[cat]

    # Best Faculty Per Category
    best_faculty_per_category = {
        category: filtered_faculty.loc[filtered_faculty[f"Predicted_{category}"].idxmax(), ['EmpId', f"Predicted_{category}"]].to_dict()
        for category in categories
    }


    weight_vector = np.array([student_prefs.get(feature, 1) for feature in categories])
    weight_vector = weight_vector / weight_vector.sum()

    faculty_characteristics = filtered_faculty[[f"Predicted_{cat}" for cat in categories]].values
    custom_scores = np.dot(faculty_characteristics, weight_vector)

    filtered_faculty["Custom_Score"] = custom_scores
    ranked_faculty = filtered_faculty.sort_values(by="Custom_Score", ascending=False)

    best_overall_faculty = ranked_faculty.iloc[0][['EmpId', 'Custom_Score']].to_dict()

    return {
        "best_per_category": best_faculty_per_category,
        "overall_best_faculty": best_overall_faculty,
        "ranked_faculties": ranked_faculty[['EmpId', 'Custom_Score']].to_dict(orient='records')
    }

# Example Usage
faculty_list = [10527, 15752, 21772]  # sample faculty IDs
student_prefs = {
    'Teaching': 8.0,
    'Lenient': 6.5,
    'DA and Quiz': 7.0,
    'Marking': 7.5
}

result = predict_best_faculty(faculty_list, student_prefs)

print("✅ Best Faculty Per Category:")
for category, faculty in result["best_per_category"].items():
    print(f"{category}: Faculty {faculty['EmpId']} (Score: {faculty[f'Predicted_{category}']:.2f})")

print("\n🏆 Overall Best Faculty:")
print(f"Faculty {result['overall_best_faculty']['EmpId']} (Custom Score: {result['overall_best_faculty']['Custom_Score']:.2f})")

print("\n📜 Full Faculty Ranking:")
for faculty in result["ranked_faculties"]:
    print(f"Faculty {faculty['EmpId']} - Score: {faculty['Custom_Score']:.2f}")


✅ Best Faculty Per Category:
Teaching: Faculty 15752.0 (Score: 8.23)
Lenient: Faculty 21772.0 (Score: 7.28)
DA and Quiz: Faculty 10527.0 (Score: 8.09)
Marking: Faculty 21772.0 (Score: 7.82)

🏆 Overall Best Faculty:
Faculty 10527.0 (Custom Score: 7.38)

📜 Full Faculty Ranking:
Faculty 10527 - Score: 7.38
Faculty 15752 - Score: 7.18
Faculty 21772 - Score: 7.14


In [ ]:
import itertools

def is_clash(existing_timetable, subject_slots):
    """Checks if the proposed subject slots clash with the existing timetable."""
    for slot in subject_slots:
        if slot in existing_timetable:
            return True
    return False

def assign_best_faculty_and_slots(subjects, student_prefs):
    """
    Assigns the best faculty for each subject while ensuring no timetable clashes.
    Returns a clash-free optimized timetable.
    """
    optimized_timetable = {}
    faculty_assignments = {}

    for subject in subjects:
        subject_name = subject["subject_name"]
        course_type = subject["course_type"]
        available_faculties = subject["available_faculties"]


        result = predict_best_faculty(available_faculties, student_prefs)
        best_faculty = result["overall_best_faculty"]["EmpId"]


        if course_type == "Theory Only":
            for slot in subject["available_slots"]:
                if not is_clash(optimized_timetable, [slot]):
                    optimized_timetable[slot] = {"subject": subject_name, "faculty": best_faculty}
                    faculty_assignments[subject_name] = best_faculty
                    break

        elif course_type == "Lab Only":
            lab_slots = subject["available_slots"]
            if len(lab_slots) >= 2:
                for i in range(len(lab_slots) - 1):
                    if not is_clash(optimized_timetable, [lab_slots[i], lab_slots[i + 1]]):
                        optimized_timetable[lab_slots[i]] = optimized_timetable[lab_slots[i + 1]] = {
                            "subject": subject_name, "faculty": best_faculty
                        }
                        faculty_assignments[subject_name] = best_faculty
                        break

        elif course_type == "Theory + Lab":
            theory_slots = subject["available_slots"]["theory"]
            lab_slots = subject["available_slots"]["lab"]

            assigned = False
            for t_slot, l_slot in itertools.product(theory_slots, range(len(lab_slots) - 1)):
                if not is_clash(optimized_timetable, [t_slot, lab_slots[l_slot], lab_slots[l_slot + 1]]):
                    optimized_timetable[t_slot] = {"subject": subject_name, "faculty": best_faculty}
                    optimized_timetable[lab_slots[l_slot]] = optimized_timetable[lab_slots[l_slot + 1]] = {
                        "subject": subject_name, "faculty": best_faculty
                    }
                    faculty_assignments[subject_name] = best_faculty
                    assigned = True
                    break

            if not assigned:
                print(f"⚠️ Could not assign a clash-free slot for {subject_name}")

    return optimized_timetable, faculty_assignments




student_prefs = {'Teaching': 8.0, 'Lenient': 6.5, 'DA and Quiz': 7.0, 'Marking': 7.5}
optimized_timetable, faculty_assignments = assign_best_faculty_and_slots(user_subjects, student_prefs)

print("\n📅 Clash-Free Timetable:")
for slot, details in optimized_timetable.items():
    print(f"{slot}: {details['subject']} (Faculty {details['faculty']})")

print("\n✅ Faculty Assignments:")
for subject, faculty in faculty_assignments.items():
    print(f"{subject}: Faculty {faculty}")



📅 Clash-Free Timetable:
A1: Mathematics (Faculty 10527.0)
C1: Physics (Faculty 10527.0)
L1: Physics (Faculty 10527.0)
L2: Physics (Faculty 10527.0)
L3: Chemistry (Faculty 15752.0)
L4: Chemistry (Faculty 15752.0)

✅ Faculty Assignments:
Mathematics: Faculty 10527.0
Physics: Faculty 10527.0
Chemistry: Faculty 15752.0
